In [1]:
'''
Obtiene letra de canciones
'''
import cancion_logica, letras_logica

# obtener canciones pendientes si letras registradas
canciones_pendientes = cancion_logica.obtenerCancionesPendientes()

for cancion_pendiente in canciones_pendientes:
   # obtiene el nombre de la canción y del cantante de la base de datos que no tiene letras
   nombre_cancion = cancion_pendiente['cancion']
   nombre_cantante = cancion_pendiente['cantante']
   
   # obtiene letra de canción desde Lyrics
   letra_cancion = letras_logica.obtenerLetraConLyrics(nombre_cancion, nombre_cantante)
   if letra_cancion == '':
      letra_cancion = letras_logica.obtenerLetraConLetras4u(nombre_cancion, nombre_cantante)
      if letra_cancion == '':
         letra_cancion = letras_logica.obtenerLetraConYaLetras(nombre_cancion, nombre_cantante)
         if letra_cancion == '':
            letra_cancion = letras_logica.obtenerLetraConCancioneros(nombre_cancion, nombre_cantante)
            if letra_cancion == '':
               letra_cancion = letras_logica.obtenerLetraConGoogle(nombre_cancion, nombre_cantante)
      
   # actualiza la letra en la colección de la canción
   resultado_final = cancion_logica.actualizarCancionPendiente(cancion_pendiente, letra_cancion)
   print(resultado_final)

------------------
cancion:  nueva%20york%20tot
cantante:  bad%20gyal
URL búsqueda en Lyrics:  https://www.lyrics.com/lyrics/nueva%20york%20tot%20bad%20gyal
------------------
Revisar:  [<div class="sec-lyric clearfix">
<div class="lyric-meta within-lyrics fll">
<div>
<p class="lyric-meta-title"><a href="/lyric-lf/6720102/Bad+Gyal/Nueva+York+%28Tot%2A%29">Nueva York (Tot*)</a></p>
<p class="lyric-meta-album-artist"><a href="artist/Bad-Gyal/3415807">Bad Gyal</a></p>
</div>
<div class="clearfix"></div>
<div>
</div>
</div>
<pre class="lyric-body" onclick="location.href='https://www.lyrics.com/lyric-lf/6720102/Bad+Gyal/Nueva+York+%28Tot%2A%29';"><em>Bad</em> <em>Gyal</em>
De Barcelona a <em>Nueva</em> <em>york</em>, la que vende todo
De Barcelona a <em>Nueva</em> <em>york</em>, no queda na'
Fumamo' adentro, nos da igual, quemándono' el hash</pre>
</div>, <div class="sec-lyric clearfix">
<div class="lyric-meta within-lyrics fll">
<div>
<p class="lyric-meta-title"><a href="/lyric-lf/8198810/